In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [2]:
# Cria sessão Spark com configs para MinIO
spark = SparkSession.builder \
    .appName("SPTransPipeline") \
    .master("spark://spark:7077") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()


# Desativar criação do arquivo _SUCCESS
spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

# Inicializando a sessão Spark
spark = SparkSession.builder.appName("RenomearColunas").getOrCreate()

In [22]:
# Leitura das tabelas silver
pv = spark.read.parquet("s3a://sptrans-data/silver/posicao")
ls = spark.read.parquet("s3a://sptrans-data/silver/linhas")

# Renomear colunas antes do join
pv = pv.withColumnRenamed("sentido", "sentido_pv")

# Fazer o join usando as colunas renomeadas
df = pv.join(
    ls,
    (pv["codigo_linha"] == ls["LetreiroCompleto"]) &
    (pv["sentido_pv"]   == ls["Sentido"]),
    "inner"
)
colunas_final = ["codigo_linha","LetreiroCompleto", "DescricaoCompleto","Sentido","TipoLinhaDescricao","LinhaCircular","latitude","longitude", "acessivel", "periodo_do_dia", "dia_semana", "hr_atualizacao"]

df = df.select(colunas_final)

# Definir a janela
windowSpec = Window.partitionBy("DescricaoCompleto", "Sentido").orderBy(F.col("hr_atualizacao").desc())

# Adicionar ranking
df_ranked = df.withColumn("rank", F.row_number().over(windowSpec))

# Filtrar apenas o último registro
vw_ultima_posicao_linha = df_ranked.filter(F.col("rank") == 1).drop("rank")


In [27]:
# Definir janela para pegar última posição
windowSpec = Window.partitionBy("DescricaoCompleto", "Sentido").orderBy(F.col("hr_atualizacao").desc())
df_ranked = df.withColumn("rank", F.row_number().over(windowSpec))
vw_ultima_posicao_linha = df_ranked.filter(F.col("rank") == 1).drop("rank")

# Caminho da tabela Delta
delta_path = "s3a://sptrans-data/gold/ultima_posicao"

# Se a tabela já existe, faz MERGE (upsert)
if DeltaTable.isDeltaTable(spark, delta_path):
    delta_table = DeltaTable.forPath(spark, delta_path)
    (
        delta_table.alias("t")
        .merge(
            vw_ultima_posicao_linha.alias("s"),
            "t.codigo_linha = s.codigo_linha AND t.Sentido = s.Sentido"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
else:
    vw_ultima_posicao_linha.write.format("delta").mode("overwrite").save(delta_path)


43614

In [25]:
vw_ultima_posicao_linha.show(n=10, truncate=False)

+------------+----------------+----------------------------+-------+------------------+-------------+-------------------+-------------------+---------+--------------+----------+-------------------+
|codigo_linha|LetreiroCompleto|DescricaoCompleto           |Sentido|TipoLinhaDescricao|LinhaCircular|latitude           |longitude          |acessivel|periodo_do_dia|dia_semana|hr_atualizacao     |
+------------+----------------+----------------------------+-------+------------------+-------------+-------------------+-------------------+---------+--------------+----------+-------------------+
|1012-21     |1012-21         |1012-21 - JD. ROSINHA       |Ida    |Interbairros      |Sim          |-23.432145         |-46.787099999999995|Sim      |Noite         |Domingo   |2025-11-23 21:59:36|
|1016-10     |1016-10         |1016-10 - CEM. DO HORTO     |Volta  |Radial            |Nao          |-23.470093         |-46.6115595        |Sim      |Noite         |Domingo   |2025-11-23 22:09:43|
|1016-10  